<a href="https://colab.research.google.com/github/Aisha-Hagar/LLMs/blob/main/TG_Data_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Setup**

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 593.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.5
    Uninstalling huggingface-hub-0.33.5:
      Successfully uninstalled huggingface-hub-0.33.5


In [2]:
from datasets import load_dataset

#**Load Data**

In [3]:
ds = load_dataset("azrai99/the-star-news-articles")
ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

news_v1.csv:   0%|          | 0.00/67.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21709 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['content_id', 'title', 'text', 'section', 'category', 'content_tier', 'content_length', 'authors', 'published_date', 'keywords', 'summary', 'url', 'top_image'],
        num_rows: 21709
    })
})

#**Data Preparation**

In [4]:
ds = ds["train"].train_test_split(test_size=0.2)
ds

DatasetDict({
    train: Dataset({
        features: ['content_id', 'title', 'text', 'section', 'category', 'content_tier', 'content_length', 'authors', 'published_date', 'keywords', 'summary', 'url', 'top_image'],
        num_rows: 17367
    })
    test: Dataset({
        features: ['content_id', 'title', 'text', 'section', 'category', 'content_tier', 'content_length', 'authors', 'published_date', 'keywords', 'summary', 'url', 'top_image'],
        num_rows: 4342
    })
})

In [5]:
ds_tst = ds["test"].train_test_split(test_size=0.5)
ds_tst

DatasetDict({
    train: Dataset({
        features: ['content_id', 'title', 'text', 'section', 'category', 'content_tier', 'content_length', 'authors', 'published_date', 'keywords', 'summary', 'url', 'top_image'],
        num_rows: 2171
    })
    test: Dataset({
        features: ['content_id', 'title', 'text', 'section', 'category', 'content_tier', 'content_length', 'authors', 'published_date', 'keywords', 'summary', 'url', 'top_image'],
        num_rows: 2171
    })
})

In [6]:
ds["validation"] = ds_tst["train"]
ds["test"] = ds_tst["test"]
ds

DatasetDict({
    train: Dataset({
        features: ['content_id', 'title', 'text', 'section', 'category', 'content_tier', 'content_length', 'authors', 'published_date', 'keywords', 'summary', 'url', 'top_image'],
        num_rows: 17367
    })
    test: Dataset({
        features: ['content_id', 'title', 'text', 'section', 'category', 'content_tier', 'content_length', 'authors', 'published_date', 'keywords', 'summary', 'url', 'top_image'],
        num_rows: 2171
    })
    validation: Dataset({
        features: ['content_id', 'title', 'text', 'section', 'category', 'content_tier', 'content_length', 'authors', 'published_date', 'keywords', 'summary', 'url', 'top_image'],
        num_rows: 2171
    })
})

In [7]:
# We only need the Article title and body, so other columns are dropped
rem_cols = ['content_id', 'section', 'category', 'content_tier', 'content_length', 'authors', 'published_date', 'keywords', 'summary', 'url', 'top_image']

In [8]:
for d in ds:
  ds[d] = ds[d].remove_columns(rem_cols)
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'text'],
        num_rows: 17367
    })
    test: Dataset({
        features: ['title', 'text'],
        num_rows: 2171
    })
    validation: Dataset({
        features: ['title', 'text'],
        num_rows: 2171
    })
})

In [9]:
def show_samples(dataset, num_samples=3, seed=42):
  """Prints random samples from the dataset.
  Args:
      dataset (Dataset): The dataset to show the samples from.
      num_samples (int): The number of samples to show. Defaults to 3.
      seed (int): The starting number to initialize the random number generator.
  """
  sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
  for example in sample:
    print(f"\n'>> Target Title: {example['title']}'")
    print(f"'>> Article: {example['text']}'")


show_samples(ds)


'>> Target Title: Johor Unduk Ngadau queen determined to chart own course despite family ties'
'>> Article: KOTA KINABALU: Romandawi Gapari may have influential family links, but the Johor Unduk Ngadau queen is determined to chart her own path.

As the final draws closer, she believes that her status as the daughter of Deputy Chief Minister Datuk Seri Dr Jeffrey Gapari Kitingan and niece of Unduk Ngadau Kaamatan organising chairman Joanna Kitingan may not necessarily work in her favour.

Additionally, the Kadazandusun Cultural Association (KDCA) president, Huguan Siou (Kadazandusun and Murut paramount leader) Tan Sri Joseph Pairin, is Jeffrey and Joanna’s elder brother.

KDCA plays a pivotal role in ensuring that the Kaamatan celebration remains faithful to the traditions and cultures of the Kadazandusun, Murut (KDM), and Rungus ethnic groups.

The state government leads in organising the festival, widely regarded as Malaysia’s largest and most significant cultural event, particularly

In [10]:
#Filter only titles of length >2
ds = ds.filter(lambda x: len(x["title"].split()) > 2)
ds

Filter:   0%|          | 0/17367 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2171 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2171 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'text'],
        num_rows: 17339
    })
    test: Dataset({
        features: ['title', 'text'],
        num_rows: 2167
    })
    validation: Dataset({
        features: ['title', 'text'],
        num_rows: 2167
    })
})

#**Save Data**

In [11]:
ds.save_to_disk("Data")

Saving the dataset (0/1 shards):   0%|          | 0/17339 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2167 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2167 [00:00<?, ? examples/s]

In [12]:
# If using colab, Uncomment the following lines to save the data to your google drive.
# from google.colab import drive
# drive.mount('/content/drive')

In [13]:
#!cp -r /content/Data "/content/drive/My Drive/Projects/Title Generator/"